# blinky-thermo-cam viewer

In [1]:
import pymongo
import datetime
import pytz
from matplotlib import pyplot
import numpy
import math
import ipywidgets as widgets
from IPython.display import display
from IPython.display import HTML
from IPython.display import clear_output
import shutil
import os

client = None

imageArrays = []
timeStampArray = []
goodData = False
thermoPlot = widgets.Output()
minPixy =  100000000
maxPixy = -100000000

def plotImage(minPix, maxPix,  frame=0):
    global minPixy
    global maxPixy
    minPixy = minPix
    maxPixy = maxPix
    pyplot.figure(figsize=(9,6))
    pyplot.imshow(imageArrays[frame][::-1,:],cmap=pyplot.get_cmap('plasma'),vmax=maxPix,vmin=minPix)
    pyplot.colorbar()
    title = "Frame:" + str(frame) + "/" + str(len(timeStampArray) - 1) + " - " + timeStampArray[frame].strftime("%b-%d-%Y, %H:%M:%S")
    pyplot.title(title)
    pyplot.tight_layout()

def plotImageToFile(frame=0):
    pyplot.figure(figsize=(9,6))
    pyplot.imshow(imageArrays[frame][::-1,:],cmap=pyplot.get_cmap('plasma'),vmax=maxPixy,vmin=minPixy)
    pyplot.colorbar()
    title = "Frame:" + str(frame) + "/" + str(len(timeStampArray) - 1) + " - " + timeStampArray[frame].strftime("%b-%d-%Y, %H:%M:%S")
    pyplot.title(title)
    pyplot.tight_layout()
    fileTitle = 'images/frame{:04d}'.format(frame)
    pyplot.savefig(fileTitle, bbox_inches='tight')
    pyplot.close()

def saveData(self):
    saveDataButton.disabled = True
    disableUI(True)
    try:
        shutil.rmtree("/home/jovyan/work/images")
    except:
        messageField.value = "Image directory does not exist. Creating."
    os.mkdir("/home/jovyan/work/images")
    messageField.value = "Saving image data."
    try:
        for iframe in range(len(imageArrays)):
            plotImageToFile(frame=iframe)
            messageField.value = messageField.value + "."
        shutil.make_archive('images', 'zip', '/home/jovyan/work/images')
        shutil.rmtree("/home/jovyan/work/images")
        saveDataButton.disabled = False
        disableUI(False)
        messageField.value = "Image data saved"
    except:
        saveDataButton.disabled = False
        disableUI(False)
        messageField.value = "Error in saving image data"
    saveDataButton.disabled = False
    disableUI(False)


def disableUI(disable):
    startDateField.disabled = disable
    stopDateField.disabled = disable
    trayNameField.disabled = disable
    databasePwField.disabled = disable
    submitPasswordButton.disabled = disable
    frameInterval.disabled = disable

def submitPasswordButtonCallback(self):
    goodData = False
    messageField.value = "Retrieving data."
    disableUI(True)
    saveDataButton.disabled = True
    client = pymongo.MongoClient("mongodb://admin:" + databasePwField.value + "@blinky-mongo:27017/blinky-lite?retryWrites=true&serverSelectionTimeoutMS=5000&connectTimeoutMS=10000&authSource=admin&authMechanism=SCRAM-SHA-256")
    startTimeStamp = (datetime.datetime(startDateField.value.year,startDateField.value.month,startDateField.value.day,startDateField.value.hour,startDateField.value.minute,startDateField.value.second) - datetime.datetime(1970, 1, 1)).total_seconds() * 1000;
    stopTimeStamp  = (datetime.datetime(stopDateField.value.year,stopDateField.value.month,stopDateField.value.day,stopDateField.value.hour,stopDateField.value.minute,stopDateField.value.second) - datetime.datetime(1970, 1, 1)).total_seconds() * 1000;
    queryFilter = {"$and" : [{"type": "blinky-thermo-cam"},{"name": trayNameField.value},{"timeStamp" : {"$gt": startTimeStamp, "$lt": stopTimeStamp} } ] }
    try:
        records = client["blinky-lite"].archiver.find(queryFilter)
        goodData = True
    except:
        messageField.value = "Failure: Images not retrieved"
    if goodData:
        while len(imageArrays) > 0:
            imageArrays.pop(0)
        while len(timeStampArray) > 0:
            timeStampArray.pop(0)
        global minPixy
        global maxPixy
        minPixy =  100000000
        maxPixy = -100000000
        for irec in records:
            try:
                imageArray = numpy.array(irec['imageData']['imageArray'])
                timeStampArray.append(datetime.datetime.fromtimestamp(round(irec['imageData']['timeStamp'] / 1000)))
                for irow in range(irec['imageData']['nrows']):
                    for icol in range(irec['imageData']['ncols']):
                        imageArray[irow][icol] = irec['imageData']['minval'] - 27315 + (irec['imageData']['maxval'] - irec['imageData']['minval']) * imageArray[irow][icol] / 255.0
                        imageArray[irow][icol] = imageArray[irow][icol] * 10
                imageArrays.append(imageArray)
                maxPixS = numpy.amax(imageArray)
                minPixS = numpy.amin(imageArray)
                if maxPixS > maxPixy:
                    maxPixy = maxPixS
                if minPixS < minPixy:
                    minPixy = minPixS
                messageField.value = messageField.value + "."
            except:
                messageField.value = "Bad record. Skipping data. Continuing to process."
            maxPixy = round(1000 * math.ceil(maxPixy / 1000),0)
            minPixy = round(1000 * math.floor(minPixy / 1000),0)
        with thermoPlot:
            clear_output(wait=True)
            if len(imageArrays) > 0:
                messageField.value = "Image retrieval completed."
                maxPixSliderSlider = widgets.IntSlider(min=minPixy, max=maxPixy, value=maxPixy, step=1000)
                minPixSliderSlider = widgets.IntSlider(min=minPixy, max=maxPixy, value=minPixy, step=1000)
                sliderLocationSlider = widgets.IntSlider(min=0, max=len(imageArrays) - 1, value=0, step=1)
                loopPlayControl = widgets.Play(min=0, max=len(imageArrays) - 1, step=1, interval=frameInterval.value)
                sliderplot = widgets.interactive(plotImage, maxPix=maxPixSliderSlider,   minPix=minPixSliderSlider, frame=sliderLocationSlider)
                loopPlot   = widgets.interactive(plotImage, maxPix=maxPixSliderSlider,   minPix=minPixSliderSlider, frame=loopPlayControl)
                accordion = widgets.Accordion(children=[sliderplot, loopPlot], titles=('Slider', 'Animation'))
                display(accordion)
                saveDataButton.disabled = False
            else:
                messageField.value = "No images found"
                saveDataButton.disabled = True
    disableUI(False)

now = pytz.utc.localize(datetime.datetime.now().replace(second=0, microsecond=0))
then = now - datetime.timedelta(days=1)
startDateField = widgets.DatetimePicker(description='Start', disabled=False, value=then)
stopDateField = widgets.DatetimePicker(description='End',disabled=False, value=now)
trayNameField = widgets.Text(value='01',placeholder='',description='Tray:',disabled=False)
databasePwField = widgets.Password(value='areallybadpassword',description='Password:',disabled=False)
submitPasswordButton = widgets.Button(description='Retrieve Data',disabled=False,button_style='danger',tooltip='Retrieve Data',icon='')
submitPasswordButton.on_click(submitPasswordButtonCallback)
frameInterval = widgets.BoundedIntText(value=1000,min=200,max=10000,step=100,description='Int.(mS):',disabled=False, continuous_update=False)
saveDataButton = widgets.Button(description='Save Data',disabled=True,button_style='warning',tooltip='Retrieve Data',icon='')
saveDataButton.on_click(saveData)
line1 =  widgets.HBox([trayNameField,frameInterval] )
line2 =  widgets.HBox([startDateField,stopDateField])
line3 =  widgets.HBox([databasePwField,submitPasswordButton])
messageField = widgets.Label(value="")
entryField = widgets.VBox([line1,line2,line3,saveDataButton,messageField])

display(entryField)
display(thermoPlot)

Output()